# Assessment 1:
## Get the square root of 2 and display to 100 decimal places

### Issue

Number of characters that can be displayed, and how to calculate the the square root of 2 to 100 decimal places.

### Option 1: Newton Raphson method

There are various options for calculating the square root of 2. The most well known is the probably the Newton Raphson method, which is similar to the Babylonian method. This method can take a small number of iterations to produce a 100 digit answer.

The method divides the number whose square root is to be found (*Number* in this case 2) by a starting number (*zStart*, initial value of 1). The resultant is then added back to zStart, and then the whole number is divided by 2. This process continues until the difference between the zStart number, and the previous zStart number (*xRoot*) is less than 1 divided by 100 to the power of 100 (i.e. there is less than digit difference at 200 decimal places).

In [57]:
import datetime as dt

Time_1 = dt.datetime.now()

Number = 2
bStart = 1                                                                  # Start value is 1
zStart = 1                                                                  # Start value is 1 (x0)
cFinal = 0                                                                  # Variable to continue/end while loop

aCount = 0                                                                  # Iteration count
yNewNum = 0                                                                 # New value of estimate (x1)
xRoot = 0                                                                   # Used to transfer values between yNewNum and zStart

while cFinal != 1:                                                          # While the difference between zStart and yNewNum is greater than 0.0000001 (7 decimal places)
    yNewNum = (zStart + (Number / zStart)) / 2                              # x1 = (x0 + (S/x0)) / 2 
    zStart = yNewNum                                                        # Replace the value x0 with the value of x1
    yNewNum = xRoot                                                         # Replace the value of x1 with the transfer value

    if abs(((zStart - xRoot) / zStart)) < (1/(100**100)):                   # When there is no significant change in the 100^100 decimal place
        cFinal = 1                                                          # Switch off continuance veriable
          
    xRoot = zStart                                                          # Replace the transfer value with x0

    aCount += 1                                                             # aCount iterates by 1

Time_2 = dt.datetime.now()

print()
print("Ne")
print("Square root 2 to 100 decimal places (Newton-Raphson method): \n %.100f" % zStart)
print()
print("Steps required to run the Newton-Raphson method to get to 100 decimal places: %d" % aCount)
print()
print("Time taken to run the Newton-Raphson method :", (Time_2-Time_1))


Square root 2 to 100 decimal places (Newton-Raphson method): 
 1.4142135623730949234300169337075203657150268554687500000000000000000000000000000000000000000000000000

Steps required to run the Newton-Raphson method to get to 100 decimal places: 6

Time taken to run the Newton-Raphson method : 0:00:00


#### Issue with the Newton-Raphson method

The calculated Newton-Raphson method diverges from the "official" NASA version after 17 decimal places, where the method returns 0949 instead od 0950. This is due to issues within Python, where floats do not generally display more than 17 decimal places, even if they hold more.

Additionally, event when the variables hold more than 17 decimal places, they cannot hold more than 50 decimal places. This results in all the digits after the 50th decimal place being printed as 0. This can be seen in the above print out, when the float was specified at 100 decimal places. This issue can be resolved by using imported libraries, which can force more decimal places to be held. However that was not allowed by the rules of the assignment.

### Option 3: Subtraction method
Source: http://www.afjarvis.staff.shef.ac.uk/maths/jarvisspec02.pdf
https://www.mathblog.dk/project-euler-80-digits-irrational-square-roots/

Square Roots by Subtraction
FRAZER JARVIS
pages 119 - 122
Mathematical Spectrum 2004/2005 Volume 37 Number 3

Japanese approach. Slower than Newton-Raphson.
Newton-Raphson takes 8 steps to run, in near instaneous time.
Subtraction method takes 576 iterations, and up to 500ths of a second to run

In [58]:
def sub_root(a, b):                                     # Create a function to import values of a and b.
    while len(str(b)) <= 102:                           # While the string length of b is less than 101 (1 integter, plus the 101 significant decimal place).

        if a >= b:                                      # If input "a" is greater than or equal input "b".
            a = a - b                                   # Set value of "a" to be "a" minus "b".
            b = b + 10                                  # Set value of "b" to be "b" + 10.

        elif a < b:                                     # If input "a" is less than input "b".
            a = a * 100                                 # Multiply "a" by 100.
            c = len(str(b))-1                           # Set the value "c" to be the string length of "b" minus 1.
            d = str(b)                                  # Set the value "d" to be the string "b".
            e = int(d[:c] + "0" + d[c:])                # Set the value of "e" to be the first "c" number of characters of the string "d".
                                                        # concatenated with a "0" and finished with the last digit of the string "d".
            b = e                                       # Set the value of "b" to be the same as "e".

        return sub_root(a, b)                           # Return the value to the function to reprocess.

    return b                                            # Return the value for "b" when the above while loop is completed.

Number = 2                                              # Number whose square root is to be determined.
SQRT = str(sub_root(Number * 5, 5))                     # Set the value of "SQRT" to be the string extracted from the function "sub_root".
formatted_SQRT = f'{SQRT[0]}.{SQRT[1:101]}'             # Set the value of "format_SQRT to be the value of "SQRT" with a decimal place after the 1st digit in the string.
print("Square root 2 to 100 decimal places:")           # Print out the text.
print(formatted_SQRT)                                   # Print the value of "format_SQRT" to 100 decimal places.

Square root 2 to 100 decimal places:
1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727


Actual square root of 2:

1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727

https://apod.nasa.gov/htmltest/gifcity/sqrt2.1mil
The Square Root of Two to 1 Million Digits
Robert Nemiroff and Jerry Bonnell 


In [59]:
def sum_digits(digit):
    return sum(int(x) for x in digit if x.isdigit())

print(sum_digits(SQRT[:100]))

# https://stackoverflow.com/a/14550071/11485706
# Óscar López
# Jan 27 '13 at 17:32

475


https://projecteuler.net/problem=80

Square root digital expansion
Show HTML problem content 
Problem 80

It is well known that if the square root of a natural number is not an integer, then it is irrational. The decimal expansion of such square roots is infinite without any repeating pattern at all.

The square root of two is 1.41421356237309504880..., and the digital sum of the first one hundred decimal digits is 475.

For the first one hundred natural numbers, find the total of the digital sums of the first one hundred decimal digits for all the irrational square roots.

1.14 Decimals, Floats, and Floating Point Arithmetic. http://anh.cs.luc.edu/python/hands-on/3.1/handsonHtml/float.html
Hands-on-Python Tutorial
© Copyright 2019, Dr. Andrew N. Harrington. Last updated on Jan 05, 2020. Created using Sphinx 1.3.1+.
October 3rd, 2020

15. Floating Point Arithmetic: Issues and Limitations
The Python Tutorial
© Copyright 2001-2020, Python Software Foundation.
The Python Software Foundation is a non-profit corporation. Please donate.

Oct 08, 2020

Period of the Continued Fraction of √n
Marius Beceanu
February 5, 2003
https://web.archive.org/web/20151221205104/http://web.math.princeton.edu/mathlab/jr02fall/Periodicity/mariusjp.pdf

https://apod.nasa.gov/htmltest/gifcity/sqrt2.1mil

In [60]:
# NASA calculated square root of 2

NASA = 1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727

Calc_SQRT = float(formatted_SQRT)

print()
print("Difference between NASA calualtions and Newton-Raphson method is: ", (zStart - NASA))
print("Percentage difference between NASA calualtions and Newton-Raphson method is: %.20f%%" % (((NASA - zStart) / NASA) * 100))

print()
print("Difference between NASA calualtions and Newton-Raphson method is: ", (Calc_SQRT - NASA))
print("Percentage difference between NASA calualtions and Newton-Raphson method is: %.20f%%" % (((NASA - Calc_SQRT) / NASA) * 100))


Difference between NASA calualtions and Newton-Raphson method is:  -2.220446049250313e-16
Percentage difference between NASA calualtions and Newton-Raphson method is: 0.00000000000001570092%

Difference between NASA calualtions and Newton-Raphson method is:  0.0
Percentage difference between NASA calualtions and Newton-Raphson method is: 0.00000000000000000000%


# Assessment 2:
## Calculate the Chi-squared test for independence for the given table

### What is Chi-Squared testing for independence


### Assignment

#### Source data

In [61]:
# Script to determine the Chi-squared test results for the given table

from scipy import stats as st
import numpy as np

# Create the lists containing the values for each of the collars
WC = [90, 60, 104, 95]                                                  # White Collar
BC = [30, 50, 51, 20]                                                   # Blue Colloar
NC = [30, 40, 45, 35]                                                   # No Collar

# Create the contingency table
collars = np.array([WC, BC, NC])                                        # Use the lists to create a 2D array

# Return the chi squared values, probability, degrees of freedom, and expected tables
chi_sqr_stat, p_val, D_of_F, expect_t = st.chi2_contingency(collars)

print("Critical value of Chi squared")
print("%.3f" %chi_sqr_stat)
print()
print("P-value")
print("%.6f" %p_val)
print()
print("Degrees of Freedom")
print(D_of_F)
print()
print("Expected Table")
print(expect_t)
print()

Critical value of Chi squared
24.571

P-value
0.000410

Degrees of Freedom
6

Expected Table
[[ 80.53846154  80.53846154 107.38461538  80.53846154]
 [ 34.84615385  34.84615385  46.46153846  34.84615385]
 [ 34.61538462  34.61538462  46.15384615  34.61538462]]



In [62]:
# Determine the Critical stat value at two levels of probability

probability_1 = 0.05                                                    # Set probability value 1 to 5%
probability_2 = 0.01                                                    # Set probability value 2 to 1%

critical_stat_1 = st.chi2.ppf(1-probability_1, D_of_F)                  # Calculate the test_stats for 6 degrees of freedom, and 95% confidence
critical_stat_2 = st.chi2.ppf(1-probability_2, D_of_F)                  # Calculate the test_stats for 6 degrees of freedom, and 99% confidence

print("At %.0f degrees of freedom and a probability of %d%%, the critical value is %.3f" % (D_of_F, (probability_1 * 100), critical_stat_1))
print("At %.0f degrees of freedom and a probability of %d%%, the critical value is %.3f" % (D_of_F, (probability_2 * 100), critical_stat_2))

At 6 degrees of freedom and a probability of 5%, the critical value is 12.592
At 6 degrees of freedom and a probability of 1%, the critical value is 16.812


### What does this mean?

1 P-value: The P-value calculated for the contingency table is 0.000410 (or 0.041%). This value is significantly below the probability vales of 0.05 (5%) and 0.01 (1%), it would suggest that the likelihood of getting such results is extremely unlikely if the null hypothesis is correct. 

2 Degrees of Freedom: The calculated degrees of freedom is 6, which is the same as the degrees of freedom on the source page.

3 Critical stat value: The critical stat  values is calculated as 24.571, and is given as 24.6 on the Wikipedia page. These results are the same, if rounding to only 1 significant decimal place is allowed. However, as noted above the critical stat values for 6 degrees of freedom, and the p-values of 0.05 and 0.01 are 12.592, abd 16.812 respectively. Given that the calculated value is significantly higher than the expected values, it would suggest that the null hypothesis is not valid.

### Overall

As per the assignment, the calculated critical stats value was found to be the same as the given value (24.571 / 24.6, as per point 3 above). In addition the p-value that was requested is 0.000410 (as per point 1 above).

The overall result of the Chi-Squared testing suggests that the null hypothesis (*"that each person's neighborhood of residence is independent of the person's occupational classification"*) should be rejected, due to a p-value significantly lower than the probablity limits, and a critical stat value considerably higher than the Chi-Squared distribution tables would allow. As such, the alternative hypothesis *that each person's neighbourhood of residence is dependent on the person's occupational classification* must hold true.

References:
https://reneshbedre.github.io/blog/chisq.html

https://en.wikipedia.org/wiki/Chi-squared_test

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html

https://machinelearningmastery.com/chi-squared-test-for-machine-learning/

https://towardsdatascience.com/running-chi-square-tests-in-python-with-die-roll-data-b9903817c51b

https://www.geeksforgeeks.org/python-pearsons-chi-square-test/

https://www.pythonfordatascience.org/chi-square-test-of-independence-python/

https://pythonfordatascienceorg.wordpress.com/chi-square-python/

https://medium.com/@alok.ranjan4/chi-square-test-using-python-7f8f14731cb

https://www.spss-tutorials.com/chi-square-independence-test/